# Random forest classifier using bleaching alert area (BAA) 

## Load dataset

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sb
import matplotlib.pyplot as plt
import pingouin as pg
#from scipy.stats import shapiro, levene, bartlett, kruskal
import scikit_posthocs as sp
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score
from itertools import combinations, permutations

# Check scikit-learn version
from sklearn import __version__
print(__version__)

In [ ]:
'''
    Load full dataset
'''
data = pd.read_csv('baa.csv')
data.head()

In [ ]:
'''
    Subset DF by SEVERITY_CODE [0,1,2,3]
'''
#data = data.dropna() # drop rows that contains NaN's 
data = data[(data.SEVERITY_CODE == 0)|(data.SEVERITY_CODE == 1)|(data.SEVERITY_CODE == 2)|(data.SEVERITY_CODE == 3)] 
#data = data[(data.YEAR >= 2015) & (data.YEAR <= 2016)] # subset a single event
#data = data[(data.YEAR >= 1997) & (data.YEAR <= 1998)] 
#list(data.columns)
data = data.dropna() # drop rows that contains NaN's
len(data)

In [ ]:
# len(data)
data.columns

In [ ]:
'''
Editable variable's model 
'''
## 
X1=data[['Cases']] # Features (dependent variable)
y=data['SEVERITY_CODE'] #labels (indipendent variable)

In [ ]:
'''
    Variance inflation factor VIF
'''
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Get variables for which to compute VIF and add intercept term
X1['Intercept'] = 1
# Compute and view VIF
vif = pd.DataFrame()
vif["variables"] = X1.columns
vif["VIF"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
# View results using print
print(vif)

# Random forest classifier

In [ ]:
'''
    Build the models
'''
model = RandomForestClassifier(n_estimators=200, random_state=10)
model.fit(X1,y)
# evaluate the model
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=10)
n_scores = cross_val_score(model, X1, y, cv=cv) #n_jobs=-1, error_score='raise'

In [ ]:
'''
    Report performance
'''
print('Cross val score: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('\n')
# Features importance
print('=== features importances ===')
fi = pd.DataFrame({'feature': list(X1.columns),
                   'importance': model.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

In [ ]:
'''
    Confusion matrix
'''
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
y_pred=model.predict(X1)
conf_mat = confusion_matrix(y, y_pred)
conf_mat_norm = confusion_matrix(y, y_pred,normalize='all')
print("=== Confusion matrix ===")
print(conf_mat)
print('\n')
print("=== Confusion matrix normalized ===")
print(conf_mat_norm)
print('\n')
print("=== Classification Report ===")
print(classification_report(y, y_pred))
print('\n')
print('=== Accuracy and Kappa ===')
print('accuracy', metrics.accuracy_score(y, y_pred))
print('\n')
print('kappa', metrics.cohen_kappa_score(y, y_pred))
print('\n')


In [ ]:
'''
    Evaluation between classifications models through "log loss"
'''
model_probs = model.predict_proba(X1)
score = log_loss(y, model_probs)


'''
    Evaluation between classifications models through "ROC_AUC"
'''
roc_value = roc_auc_score(y, model_probs, multi_class='ovo') # ovo': Computes the average AUC of all possible pairwise combinations of classes

print('=== roc_auc_score ===') 
print(roc_value)
print(' ')
print('=== log_loss_score ===') 
print(score)


In [ ]:
ax = plt.subplot()
#Heat map with annot=True to annotate cells
sb.heatmap(conf_mat, annot=True, ax = ax, fmt='d', cmap='Blues') # actual cases
#sns.heatmap(conf_mat/np.sum(conf_mat), annot=True, ax = ax, fmt='.2%', cmap='Blues') #percentage
# labels, title and ticks
ax.set_xlabel('Predicted bleaching level');ax.set_ylabel('True bleaching level'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['level 0','level 1','level 2', 'level 3']); ax.yaxis.set_ticklabels(['level 0','level 1','level 2', 'level 3'])
plt.show()
#plt.savefig('DHW_monmean_CF_a_monmean.pdf', dpi=300)

In [ ]:
comparisons = pd.DataFrame({'Real':y, 'Predictions':y_pred})
#comparisons.to_csv('comparisons.csv')
print(comparisons[['Real','Predictions']])